In [ ]:
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
input_shape = (48, 48, 3)
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

170500096/170498071 [==============================] - 11s 0us/step


In [ ]:
# resize train set
X_train_resized = []
for img in X_train:
  X_train_resized.append(np.resize(img, input_shape) / 255)
  
X_train_resized = np.array(X_train_resized)
print(X_train_resized.shape)
# resize test set
X_test_resized = []
for img in X_test:
  X_test_resized.append(np.resize(img, input_shape) / 255)
  
X_test_resized = np.array(X_test_resized)
print(X_test_resized.shape)

(50000, 48, 48, 3)
(10000, 48, 48, 3)


In [ ]:
# We build the base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
______________________________________________________________

In [ ]:
# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers: 
  layer.trainable = False
  print('Layer ' + layer.name + ' frozen.')

# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.3)(x)
x = Dense(10, activation='softmax', name='predictions')(x)
model = Model(base_model.input, x)
# We compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Layer input_1 frozen.
Layer block1_conv1 frozen.
Layer block1_conv2 frozen.
Layer block1_pool frozen.
Layer block2_conv1 frozen.
Layer block2_conv2 frozen.
Layer block2_pool frozen.
Layer block3_conv1 frozen.
Layer block3_conv2 frozen.
Layer block3_conv3 frozen.
Layer block3_pool frozen.
Layer block4_conv1 frozen.
Layer block4_conv2 frozen.
Layer block4_conv3 frozen.
Layer block4_pool frozen.
Layer block5_conv1 frozen.
Layer block5_conv2 frozen.
Layer block5_conv3 frozen.
Layer block5_pool frozen.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
____________________

In [ ]:
# We start the training
epochs = 10
batch_size = 256
# We train it
model.fit(X_train_resized, Y_train,
          batch_size=batch_size,
          validation_data=(X_test_resized, Y_test),
          epochs=epochs)

Epoch 1/10
196/196 [==============================] - 10s 49ms/step - loss: 1.7873 - accuracy: 0.3582 - val_loss: 1.6677 - val_accuracy: 0.4044
Epoch 2/10
196/196 [==============================] - 9s 46ms/step - loss: 1.6443 - accuracy: 0.4113 - val_loss: 1.6182 - val_accuracy: 0.4260
Epoch 3/10
196/196 [==============================] - 9s 47ms/step - loss: 1.5992 - accuracy: 0.4296 - val_loss: 1.5981 - val_accuracy: 0.4317
Epoch 4/10
196/196 [==============================] - 9s 46ms/step - loss: 1.5697 - accuracy: 0.4387 - val_loss: 1.5776 - val_accuracy: 0.4335
Epoch 5/10
196/196 [==============================] - 9s 46ms/step - loss: 1.5434 - accuracy: 0.4495 - val_loss: 1.5570 - val_accuracy: 0.4428
Epoch 6/10
196/196 [==============================] - 9s 47ms/step - loss: 1.5251 - accuracy: 0.4553 - val_loss: 1.5540 - val_accuracy: 0.4426
Epoch 7/10
196/196 [==============================] - 9s 48ms/step - loss: 1.5062 - accuracy: 0.4623 - val_loss: 1.5365 - val_accuracy: 0.452